In [1]:
import os
import sys
import re
import csv

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
# import pygris
import shapely.wkt

In [2]:
os.chdir('..')
abs_path = os.getcwd()

output_path = os.path.join(abs_path, 'output')
clad_omop_output_file = "clad_omop_simulated_output_4.xlsx"
clad_omop_output_path = os.path.join(output_path, clad_omop_output_file)

omop_simulated_file = "OMOP_sample_simulated_residential.csv"
omop_simulated_path = os.path.join(output_path, omop_simulated_file)

omop_full_successful_file = "OMOP_location_flagged_successful.csv"
omop_full_successful_path = os.path.join(output_path, omop_full_successful_file)

omop_full_failed_file = "OMOP_location_flagged_failed.csv"
omop_full_failed_path = os.path.join(output_path, omop_full_failed_file)

tract_folder = os.path.join(abs_path, 'data/tl_2019_us_county')
tract_file = 'tl_2019_us_county.shp'
tract_path = os.path.join(tract_folder, tract_file)

# fips_county_file =  "fips2county.tsv"
# fips_county_path = os.path.join(abs_path, 'data', fips_county_file)

In [21]:
# omop_sample = pd.read_csv(omop_simulated_path)
# fips_county = pd.read_csv(fips_county_path, sep='\t', header='infer', dtype=str, encoding='latin-1')
omop_full = pd.read_csv(omop_full_successful_path)
omop_failed = pd.read_csv(omop_full_failed_path)
omop_failed

C:\Users\bchan\AppData\Local\Temp\ipykernel_23740\2714154275.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_full = pd.read_csv(omop_full_successful_path)


,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,156,Po Box 102,NaN,Deerfield St,NJ,08313,NaN,"PO BOX 102, DEERFIELD ST, NJ 08313",39.523911,-75.236086,PO Box,NJ,FAILED DUE TO PO BOX ADDRESS
1,157,Po Box 6149,NaN,China Village,ME,04926,NaN,"PO BOX 6149, CHINA VILLAGE, ME 04926",44.481721,-69.516751,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
2,158,Po Box 555,NaN,Northeast Hbr,ME,04662,NaN,"PO BOX 555, NORTHEAST HBR, ME 04662",44.294140,-68.290211,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
3,159,Po Box 514,NaN,Bangor,ME,04402,NaN,"PO BOX 514, BANGOR, ME 04402",44.801671,-68.772141,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
4,160,Po Box 826,NaN,Presque Isle,ME,04769,NaN,"PO BOX 826, PRESQUE ISLE, ME 04769",46.681235,-68.010188,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,104057,141 Main Street,NaN,South Bound Brook,NJ,888,NaN,"141 MAIN STREET, SOUTH BOUND BROOK, NJ 08880",40.552276,-74.523419,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15709,104082,441 United States Highway 130,NaN,Hightstown,NJ,852,NaN,"441 UNITED STATES HIGHWAY 130, HIGHTSTOWN, NJ ...",40.271074,-74.539260,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15710,104083,16 Ethel Road,NaN,Edison,NJ,8817,NaN,"16 ETHEL ROAD, EDISON, NJ 08817",40.540541,-74.398598,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15711,104085,606 Dowd Avenue,NaN,Elizabeth,NJ,7201,NaN,"606 DOWD AVENUE, ELIZABETH, NJ 07201",40.666597,-74.188170,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE


# Correct Leading Zeros
* Append `0` if not 5-digit zipcode

In [6]:
def zip_leadingZeros(x):
    zip = x.rjust(5, '0')
    return zip

In [7]:
def _to_geometry(x):
    centroid = str("POINT" + " (" + str(x.longitude) + " " + str(x.latitude) + ")")

    return centroid

In [6]:
omop_full['zip'] = omop_full.apply(lambda x: zip_leadingZeros(str(x.zip)), axis=1)
# omop_sample

In [7]:
omop_full['geometry'] = omop_full.apply(lambda x: _to_geometry(x), axis=1)
# omop_sample

In [8]:
omop_full['geometry'].astype('str').apply(shapely.wkt.loads)
OMOP_gdf = gpd.GeoDataFrame(omop_full, geometry=omop_full['geometry'].apply(shapely.wkt.loads))
OMOP_gdf.crs = "EPSG:4326"
OMOP_gdf

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry
0,1,523 E Broadway,NaN,South Boston,MA,02127,NaN,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.04352 42.33547)
1,2,454 Essex St,NaN,Lawrence,MA,01840,NaN,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.16494 42.70621)
2,3,569 Broadway,NaN,Newark,NJ,07104,NaN,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS,POINT (-74.16282 40.76993)
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,NaN,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74782 42.79853)
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,NaN,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS,POINT (-70.11378 44.42861)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78795,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,NaN,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517 34.04173)
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,NaN,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10132 33.96680)
78797,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,NaN,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90222 34.06883)
78798,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,NaN,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49411 37.99651)


# Spatial Join to TIGER/Line County

In [9]:
tract_shapes = gpd.read_file(tract_path)
tract_shapes = tract_shapes.to_crs("EPSG:4326")

In [10]:
# left join on county file
OMOP_county = OMOP_gdf.sjoin(tract_shapes, how="left")

In [11]:
OMOP_county.columns

Index(['Location_id', 'address_1', 'address_2', 'city', 'state', 'zip',
       'county', 'location_source_value', 'latitude', 'longitude',
       'address_type', 'state_abbr', 'flag', 'geometry', 'index_right',
       'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON'],
      dtype='object')

In [12]:
# fill county column with NAME from TIGER/Line for county names
OMOP_county.county = OMOP_county.NAME

# drop unncessary columns from spatial join
OMOP_county = OMOP_county.drop(columns=['geometry', 'index_right',
       'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON'])

In [13]:
OMOP_county

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,NaN,South Boston,MA,02127,Suffolk,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,NaN,Lawrence,MA,01840,Essex,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,NaN,Newark,NJ,07104,Essex,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,NaN,Orchard Park,NY,14127,Erie,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,NaN,Livermore Fls,ME,04254,Androscoggin,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78795,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,GA,30024,Gwinnett,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,GA,30096,Gwinnett,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
78797,104100,3685 Braselton Highway,NaN,Dacula,GA,30019,Gwinnett,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
78798,104101,1055 Dove Run Road,NaN,Lexington,KY,40502,Fayette,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [12]:
# Dictionary mapping state and territory abbreviations to full names
abbrev_to_us_state = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming",
    "DC": "District of Columbia",
    "AS": "American Samoa",
    "GU": "Guam",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico",
    "UM": "United States Minor Outlying Islands",
    "VI": "U.S. Virgin Islands"
}

In [13]:
def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

In [16]:
OMOP_county['state'] = OMOP_county.state_abbr.apply(lambda x: multipleReplace(str(x).strip(), abbrev_to_us_state))
OMOP_county

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78795,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS
78798,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS


In [18]:
OMOP_county_full_path = os.path.join(abs_path, 'output', 'OMOP_county_full.csv')
OMOP_county.to_csv(OMOP_county_full_path, index=False)

## County Match for Failed Addresses

In [22]:
omop_failed['zip'] = omop_failed.apply(lambda x: zip_leadingZeros(str(x.zip)), axis=1)
omop_failed['geometry'] = omop_failed.apply(lambda x: _to_geometry(x), axis=1)

# convert from pandas to geopandas dataframe
omop_failed['geometry'].astype('str').apply(shapely.wkt.loads)
OMOP_failed_gdf = gpd.GeoDataFrame(omop_failed, geometry=omop_failed['geometry'].apply(shapely.wkt.loads))
OMOP_failed_gdf.crs = "EPSG:4326"

In [23]:
# left join on county file
OMOP_failed_county = OMOP_failed_gdf.sjoin(tract_shapes, how="left")
# fill county column with NAME from TIGER/Line for county names
OMOP_failed_county.county = OMOP_failed_county.NAME

# drop unncessary columns from spatial join
OMOP_failed_county = OMOP_failed_county.drop(columns=['geometry', 'index_right',
       'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON'])
OMOP_failed_county

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,156,Po Box 102,NaN,Deerfield St,NJ,08313,Cumberland,"PO BOX 102, DEERFIELD ST, NJ 08313",39.523911,-75.236086,PO Box,NJ,FAILED DUE TO PO BOX ADDRESS
1,157,Po Box 6149,NaN,China Village,ME,04926,Kennebec,"PO BOX 6149, CHINA VILLAGE, ME 04926",44.481721,-69.516751,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
2,158,Po Box 555,NaN,Northeast Hbr,ME,04662,Hancock,"PO BOX 555, NORTHEAST HBR, ME 04662",44.294140,-68.290211,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
3,159,Po Box 514,NaN,Bangor,ME,04402,Penobscot,"PO BOX 514, BANGOR, ME 04402",44.801671,-68.772141,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
4,160,Po Box 826,NaN,Presque Isle,ME,04769,Aroostook,"PO BOX 826, PRESQUE ISLE, ME 04769",46.681235,-68.010188,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,104057,141 Main Street,NaN,South Bound Brook,NJ,00888,Somerset,"141 MAIN STREET, SOUTH BOUND BROOK, NJ 08880",40.552276,-74.523419,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15709,104082,441 United States Highway 130,NaN,Hightstown,NJ,00852,Mercer,"441 UNITED STATES HIGHWAY 130, HIGHTSTOWN, NJ ...",40.271074,-74.539260,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15710,104083,16 Ethel Road,NaN,Edison,NJ,08817,Middlesex,"16 ETHEL ROAD, EDISON, NJ 08817",40.540541,-74.398598,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15711,104085,606 Dowd Avenue,NaN,Elizabeth,NJ,07201,Union,"606 DOWD AVENUE, ELIZABETH, NJ 07201",40.666597,-74.188170,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE


In [24]:
OMOP_failed_county['state'] = OMOP_failed_county.state_abbr.apply(lambda x: multipleReplace(str(x).strip(), abbrev_to_us_state))
OMOP_failed_county

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag
0,156,Po Box 102,NaN,Deerfield St,New Jersey,08313,Cumberland,"PO BOX 102, DEERFIELD ST, NJ 08313",39.523911,-75.236086,PO Box,NJ,FAILED DUE TO PO BOX ADDRESS
1,157,Po Box 6149,NaN,China Village,Maine,04926,Kennebec,"PO BOX 6149, CHINA VILLAGE, ME 04926",44.481721,-69.516751,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
2,158,Po Box 555,NaN,Northeast Hbr,Maine,04662,Hancock,"PO BOX 555, NORTHEAST HBR, ME 04662",44.294140,-68.290211,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
3,159,Po Box 514,NaN,Bangor,Maine,04402,Penobscot,"PO BOX 514, BANGOR, ME 04402",44.801671,-68.772141,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
4,160,Po Box 826,NaN,Presque Isle,Maine,04769,Aroostook,"PO BOX 826, PRESQUE ISLE, ME 04769",46.681235,-68.010188,PO Box,ME,FAILED DUE TO PO BOX ADDRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,104057,141 Main Street,NaN,South Bound Brook,New Jersey,00888,Somerset,"141 MAIN STREET, SOUTH BOUND BROOK, NJ 08880",40.552276,-74.523419,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15709,104082,441 United States Highway 130,NaN,Hightstown,New Jersey,00852,Mercer,"441 UNITED STATES HIGHWAY 130, HIGHTSTOWN, NJ ...",40.271074,-74.539260,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15710,104083,16 Ethel Road,NaN,Edison,New Jersey,08817,Middlesex,"16 ETHEL ROAD, EDISON, NJ 08817",40.540541,-74.398598,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE
15711,104085,606 Dowd Avenue,NaN,Elizabeth,New Jersey,07201,Union,"606 DOWD AVENUE, ELIZABETH, NJ 07201",40.666597,-74.188170,Street Address,NJ,FAILED DUE TO NON 5-DIGIT ZIPCODE


In [25]:
OMOP_failed_county_full_path = os.path.join(abs_path, 'output', 'OMOP_failed_county.csv')
OMOP_failed_county.to_csv(OMOP_failed_county_full_path, index=False)

# Spatial Cross Map

In [4]:
OMOP_county = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_county_full.csv'))

C:\Users\bchan\AppData\Local\Temp\ipykernel_3876\3778721173.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  OMOP_county = pd.read_csv(os.path.join(abs_path, 'output', 'OMOP_county_full.csv'))


In [22]:
crossmap_file = "1-1_ZIP_COUNTY_HUD_file.csv"
crossmap_path = os.path.join(abs_path, "data", crossmap_file)

crossmap = pd.read_csv(crossmap_path)
crossmap.ZIP = crossmap.apply(lambda x: zip_leadingZeros(str(x.ZIP)), axis=1)
crossmap.COUNTY = crossmap.apply(lambda x: zip_leadingZeros(str(x.COUNTY)), axis=1)
crossmap

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county
0,00501,36103,0.000000,1.000000,0.000000,1.000000,NaN,single ZIP to COUNTY association,36103,36103
1,00601,72001,0.839799,0.800983,0.871166,0.838115,72001.0,centroid intersects COUNTY,72001,72001
2,00601,72113,0.160201,0.199017,0.128834,0.161885,72001.0,centroid intersects COUNTY,72001,72001
3,00602,72003,1.000000,0.998801,1.000000,0.999919,72003.0,geometry within COUNTY,72003,72003
4,00602,72005,0.000000,0.001199,0.000000,0.000081,72003.0,geometry within COUNTY,72003,72003
...,...,...,...,...,...,...,...,...,...,...
54189,99925,02198,0.000000,0.000000,1.000000,1.000000,2198.0,geometry within COUNTY,2198,2198
54190,99926,02198,0.000000,0.000000,1.000000,1.000000,2198.0,centroid intersects COUNTY,2198,2198
54191,99927,02198,0.000000,0.000000,1.000000,1.000000,2198.0,geometry within COUNTY,2198,2198
54192,99928,02130,0.000000,0.000000,1.000000,1.000000,NaN,single ZIP to COUNTY association,2130,2130


In [19]:
tract_shapes['COUNTY'] = tract_shapes.agg(lambda x: x['STATEFP'] + x['COUNTYFP'], axis=1)
tract_shapes

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,COUNTY
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",31039
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43640 46.23819, -123.44759 46.2...",53069
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99756, -104.56772 33.9...",35011
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790...",31109
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089...",31129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,13,123,00351260,13123,Gilmer,Gilmer County,06,H1,G4020,None,None,None,A,1103886891,12283371,+34.6905055,-084.4546244,"POLYGON ((-84.65478 34.66559, -84.65488 34.669...",13123
3229,27,135,00659513,27135,Roseau,Roseau County,06,H1,G4020,None,None,None,A,4329471929,16913419,+48.7610683,-095.8215042,"POLYGON ((-96.40467 48.80529, -96.40468 48.813...",27135
3230,28,089,00695768,28089,Madison,Madison County,06,H1,G4020,298,27140,None,A,1850602740,71274123,+32.6343703,-090.0341603,"POLYGON ((-90.09363 32.70762, -90.09360 32.707...",28089
3231,48,227,01383899,48227,Howard,Howard County,06,H1,G4020,None,13700,None,A,2333039143,8841783,+32.3034298,-101.4387208,"POLYGON ((-101.69227 32.27106, -101.69221 32.2...",48227


In [23]:
crossmap_tracts = crossmap.merge(tract_shapes, how='inner', on='COUNTY')
crossmap_tracts

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county,...,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,00501,36103,0.000000,1.0,0.0,1.000000,NaN,single ZIP to COUNTY association,36103,36103,...,G4020,408,35620,35004,A,2361568402,3784825485,+40.9435540,-072.6922183,"POLYGON ((-72.86371 41.11338, -72.86351 41.113..."
1,06390,36103,0.000000,1.0,1.0,1.000000,36103.0,geometry within COUNTY,36103,36103,...,G4020,408,35620,35004,A,2361568402,3784825485,+40.9435540,-072.6922183,"POLYGON ((-72.86371 41.11338, -72.86351 41.113..."
2,11701,36103,0.994771,1.0,1.0,0.995617,36103.0,geometry within COUNTY,36103,36103,...,G4020,408,35620,35004,A,2361568402,3784825485,+40.9435540,-072.6922183,"POLYGON ((-72.86371 41.11338, -72.86351 41.113..."
3,11702,36103,1.000000,1.0,1.0,1.000000,36103.0,geometry within COUNTY,36103,36103,...,G4020,408,35620,35004,A,2361568402,3784825485,+40.9435540,-072.6922183,"POLYGON ((-72.86371 41.11338, -72.86351 41.113..."
4,11703,36103,1.000000,1.0,1.0,1.000000,36103.0,geometry within COUNTY,36103,36103,...,G4020,408,35620,35004,A,2361568402,3784825485,+40.9435540,-072.6922183,"POLYGON ((-72.86371 41.11338, -72.86351 41.113..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54170,99922,02198,0.000000,0.0,1.0,1.000000,2198.0,geometry within COUNTY,2198,2198,...,G4020,None,None,None,S,13633096724,14622411293,+55.6827733,-133.1623885,"MULTIPOLYGON (((-131.71335 55.09900, -131.7036..."
54171,99923,02198,0.000000,0.0,1.0,1.000000,2198.0,geometry within COUNTY,2198,2198,...,G4020,None,None,None,S,13633096724,14622411293,+55.6827733,-133.1623885,"MULTIPOLYGON (((-131.71335 55.09900, -131.7036..."
54172,99925,02198,0.000000,0.0,1.0,1.000000,2198.0,geometry within COUNTY,2198,2198,...,G4020,None,None,None,S,13633096724,14622411293,+55.6827733,-133.1623885,"MULTIPOLYGON (((-131.71335 55.09900, -131.7036..."
54173,99926,02198,0.000000,0.0,1.0,1.000000,2198.0,centroid intersects COUNTY,2198,2198,...,G4020,None,None,None,S,13633096724,14622411293,+55.6827733,-133.1623885,"MULTIPOLYGON (((-131.71335 55.09900, -131.7036..."


In [32]:
OMOP_county['ZIP'] = OMOP_county.zip
OMOP_crossmap = OMOP_county.merge(crossmap_tracts, how='left', on='ZIP')
OMOP_crossmap

,Unnamed: 0,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,...,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,...,G4020,148,14460,14454,N,1.508631e+08,160514693.0,+42.3385513,-071.0182530,"POLYGON ((-71.03318 42.41421, -71.03301 42.414..."
1,1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,...,G4020,148,14460,15764,N,1.275609e+09,870172223.0,+42.6427082,-070.8649085,"POLYGON ((-71.18240 42.79433, -71.18237 42.794..."
2,2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex,"569 BROADWAY, NEWARK, NJ 07104",40.769935,...,G4020,408,35620,35084,A,3.265575e+08,8638174.0,+40.7874003,-074.2462919,"POLYGON ((-74.32916 40.83856, -74.32899 40.839..."
3,3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,...,G4020,160,15380,None,A,2.700649e+09,476891055.0,+42.7527590,-078.7781922,"POLYGON ((-78.95688 42.83271, -78.93056 42.850..."
4,4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,...,G4020,438,30340,None,A,1.211991e+09,75546586.0,+44.1676811,-070.2074347,"POLYGON ((-70.32522 44.20840, -70.32188 44.217..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104449,82363,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
104450,82364,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
104451,82365,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
104452,82366,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [35]:
pd.crosstab(OMOP_crossmap.county, OMOP_crossmap.NAME)

NAME,Abbeville,Acadia,Accomack,Ada,Adair,Adams,Addison,Aguadilla,Aiken,Aitkin,...,Yellowstone,Yoakum,Yolo,York,Young,Yuba,Yuma,Zapata,Zavala,Ziebach
county,,,,,,,,,,,,,,,,,,,,,
Abbeville,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acadia,0,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Accomack,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ada,0,0,0,234,0,4,0,0,0,0,...,9,0,0,0,0,0,0,0,0,0
Adair,0,0,0,0,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yuba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,0,0,0
Yuma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,103,0,0,0
Zapata,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [45]:
OMOP_crossmap['county_match'] = OMOP_crossmap.apply(lambda x: 1 if x.county == x.NAME else 0, axis=1)
OMOP_crossmap.county.isna().sum()
# OMOP_crossmap.NAME.isna().sum()

5

# RUCA County Match

In [46]:
RUCA_folder = os.path.join(abs_path, 'data')
RUCA_file = 'ruca2010revised.xlsx'
RUCA_path = os.path.join(RUCA_folder, RUCA_file)

In [50]:
RUCA_df = pd.read_excel(RUCA_path, sheet_name='Data', engine="openpyxl", header=1)
RUCA_df = RUCA_df.rename(columns={'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)':'State-County-Tract FIPS Code'})
RUCA_df['COUNTY'] = RUCA_df.apply(lambda x: zip_leadingZeros(str(x["State-County FIPS Code"])), axis=1)
RUCA_df

,State-County FIPS Code,Select State,Select County,State-County-Tract FIPS Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",COUNTY
0,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001
1,1001,AL,Autauga County,1001020200,1,1.0,2170,1.289776,1682.462370,01001
2,1001,AL,Autauga County,1001020300,1,1.0,3373,2.065366,1633.124331,01001
3,1001,AL,Autauga County,1001020400,1,1.0,4386,2.464376,1779.760676,01001
4,1001,AL,Autauga County,1001020500,1,1.0,10766,4.400686,2446.436531,01001
...,...,...,...,...,...,...,...,...,...,...
73997,72153,PR,Yauco Municipio,72153750501,1,1.0,4960,0.693337,7153.810353,72153
73998,72153,PR,Yauco Municipio,72153750502,1,1.0,2092,0.263530,7938.381391,72153
73999,72153,PR,Yauco Municipio,72153750503,1,1.0,2357,1.280621,1840.512771,72153
74000,72153,PR,Yauco Municipio,72153750601,1,1.0,5315,4.251936,1250.018953,72153


In [52]:
RUCA_tracts = RUCA_df.merge(tract_shapes, how='left', on='COUNTY')
RUCA_crossmap = RUCA_tracts.merge(crossmap, how='left', on='COUNTY')
RUCA_crossmap

,State-County FIPS Code,Select State,Select County,State-County-Tract FIPS Code,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",COUNTY,...,geometry,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Spatial_Overlap_COUNTY,overlay,MaxResCounty,Preferred_county
0,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001,...,"POLYGON ((-86.90309 32.54062, -86.90311 32.540...",36003,1.000000,1.000000,1.000000,1.000000,1001.0,geometry within COUNTY,1001.0,1001.0
1,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001,...,"POLYGON ((-86.90309 32.54062, -86.90311 32.540...",36006,0.712692,0.529412,0.555556,0.706595,1001.0,centroid intersects COUNTY,1001.0,1001.0
2,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001,...,"POLYGON ((-86.90309 32.54062, -86.90311 32.540...",36008,1.000000,1.000000,0.000000,1.000000,NaN,single ZIP to COUNTY association,1001.0,1001.0
3,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001,...,"POLYGON ((-86.90309 32.54062, -86.90311 32.540...",36022,0.363480,0.263636,0.350000,0.361758,1051.0,centroid intersects COUNTY,1051.0,1051.0
4,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727,01001,...,"POLYGON ((-86.90309 32.54062, -86.90311 32.540...",36051,0.651982,0.800000,1.000000,0.659830,1001.0,centroid intersects COUNTY,1001.0,1001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5069942,72153,PR,Yauco Municipio,72153750503,1,1.0,2357,1.280621,1840.512771,72153,...,"POLYGON ((-66.91602 18.09603, -66.91601 18.096...",00698,0.997003,0.963356,0.974843,0.994634,72153.0,centroid intersects COUNTY,72153.0,72153.0
5069943,72153,PR,Yauco Municipio,72153750601,1,1.0,5315,4.251936,1250.018953,72153,...,"POLYGON ((-66.91602 18.09603, -66.91601 18.096...",00653,0.004885,0.000000,0.000000,0.004597,72055.0,geometry within COUNTY,72055.0,72055.0
5069944,72153,PR,Yauco Municipio,72153750601,1,1.0,5315,4.251936,1250.018953,72153,...,"POLYGON ((-66.91602 18.09603, -66.91601 18.096...",00698,0.997003,0.963356,0.974843,0.994634,72153.0,centroid intersects COUNTY,72153.0,72153.0
5069945,72153,PR,Yauco Municipio,72153750602,1,1.0,3141,6.767033,464.162044,72153,...,"POLYGON ((-66.91602 18.09603, -66.91601 18.096...",00653,0.004885,0.000000,0.000000,0.004597,72055.0,geometry within COUNTY,72055.0,72055.0


# ZCTA County Match

In [ ]:
zcta_folder = os.path.join(abs_path, 'data/tl_2023_us_zcta520_clean')
zcta_file = 'tl_2023_us_zcta520_clean.shp'
zcta_path = os.path.join(zcta_folder, zcta_file)

OMOP_county_full_path = os.path.join(abs_path, 'output', 'OMOP_county_full.csv')

zcta_crosswalk_file = "tab20_zcta520_county20_natl.txt"
zcta_crosswalk_path = os.path.join(abs_path, "data", zcta_crosswalk_file)

In [ ]:
# load full omop county file
omop_county_full = pd.read_csv(OMOP_county_full_path)

# load zcta county crosswalk file
zcta_crosswalk = pd.read_csv(zcta_crosswalk_path, sep="|")

In [ ]:
# drop unnecessary columns
zcta_crosswalk = zcta_crosswalk.drop(columns=["OID_ZCTA5_20", "AREALAND_ZCTA5_20", "AREAWATER_ZCTA5_20", "MTFCC_ZCTA5_20", "CLASSFP_ZCTA5_20", "FUNCSTAT_ZCTA5_20", 
                                              "AREALAND_COUNTY_20", "AREAWATER_COUNTY_20", "MTFCC_COUNTY_20", "CLASSFP_COUNTY_20", "FUNCSTAT_COUNTY_20", "AREALAND_PART", "AREAWATER_PART"])
zcta_crosswalk

In [ ]:
# laod zcta shapefile
zcta_shapes = gpd.read_file(zcta_path)

# convert to EPSG:4326
zcta_shapes = zcta_shapes.to_crs("EPSG:4326")
zcta_shapes

In [ ]:
# save ZCTA shapefile after dropping columns
# ONLY RUN ONCE
zcta_shapes = zcta_shapes.drop(columns=["CLASSFP20", "MTFCC20", "FUNCSTAT20", "ALAND20", "AWATER20"])

# shp_file = dict[fname].set_geometry('source_centroid')
save_dir = os.path.join(abs_path, 'data')        
save_path = os.path.join(save_dir, "tl_2023_us_zcta520_clean")
create_dir(save_path)

zcta_shapes.to_file(save_path, driver='ESRI Shapefile')

In [ ]:
# convert to string without trailing zeros and decimal
zcta_crosswalk['ZCTA5CE20'] = zcta_crosswalk.GEOID_ZCTA5_20.apply(lambda x: str(x).rstrip(".0") if ".0" in str(x) else str(x))
zcta_crosswalk

In [ ]:
# merge shapefile to county crosswalk on ZCTA ID
zcta_county_name = zcta_shapes.merge(zcta_crosswalk, how="left", on="ZCTA5CE20")
zcta_county_name['zip'] = zcta_county_name.ZCTA5CE20
zcta_county_name

In [ ]:
# add point geometry column that matches expected gpd format
omop_county_full['geometry'] = omop_county_full.apply(lambda x: "POINT (" + str(x.longitude) + " " + str(x.latitude) + ")", axis=1)

# convert to geopandas DataFrame and set `geometry` centroid 
omop_county_gdf = gpd.GeoDataFrame(omop_county_full, geometry=omop_county_full['geometry'].apply(shapely.wkt.loads))
omop_county_gdf.crs = "EPSG:4326"
omop_county_gdf

In [ ]:
# spatial join omop county file to zcta
omop_county_zcta = omop_county_gdf.sjoin(zcta_county_name, how='left')

# add "County" to county names
omop_county_zcta["county"] =  omop_county_zcta.county.apply(lambda x: str(x) + " County")
omop_county_zcta

In [ ]:
# if county match label as 1, else 0
omop_county_zcta["county_match"] = omop_county_zcta.apply(lambda x: 1 if x.county == x.NAMELSAD_COUNTY_20 else 0, axis=1)
omop_county_zcta

In [ ]:
omop_county_zcta.county_match.value_counts()

In [ ]:
# save file
OMOP_county_zcta_path = os.path.join(abs_path, 'output', 'OMOP_county_zcta.csv')
omop_county_zcta.to_csv(OMOP_county_zcta_path, index=False)